In [167]:
import heapq
import sys
from typing import Self

sys.path.append("src")

# Study materials

## R-Trees

An R-tree is a hierarchical data structure designed for efficiently indexing multi-dimensional spatial objects like points, rectangles, and polygons. It works by organizing spatial data into nested, overlapping bounding boxes (rectangles) arranged in a balanced tree structure. Each node in the tree represents a bounding rectangle that contains all its children - leaf nodes store actual spatial objects while internal nodes store the minimum bounding rectangle (MBR) that encompasses all rectangles in their subtree. When searching for objects in a specific area, the algorithm starts at the root and recursively traverses only the branches whose bounding boxes intersect with the query region, dramatically reducing the search space. This enables sub-millisecond queries on millions of geographic objects, making R-trees essential for applications like finding nearby points of interest, spatial joins, and real-time map rendering.

### Pseudo-code

```
INSERT(tree, x, y, data)
```

- Check if tree is empty
- If tree.root == null:
    - Create new leaf node with max_entries capacity
  - Add entry (Rectangle(x,y,x,y), data) to node
  - Set tree.root = node
  - Set tree.height = 1
  - Return
- Find appropriate leaf node
- Call CHOOSE_LEAF(tree.root, Rectangle(x,y,x,y)) → returns leaf node L
- Add entry to leaf
- Add entry (Rectangle(x,y,x,y), data) to L.entries
- Handle potential overflow
- Call HANDLE_OVERFLOW(L)
- Update MBRs upward
- Call UPDATE_MBR_UPWARD(L)

---

```
CHOOSE_LEAF(node, point_rectangle)
```

- If node is leaf
- Return node
- If node is internal
- Initialize best_child = null, min_enlargement = infinity, min_area = infinity
- For each entry (mbr, child) in node.entries:
    - Calculate enlargement = mbr.enlargement_needed(point_rectangle)
  - Calculate area = mbr.area()
  - If enlargement < min_enlargement OR (enlargement == min_enlargement AND area < min_area):
        - Set min_enlargement = enlargement
    - Set min_area = area
    - Set best_child = child
- Return CHOOSE_LEAF(best_child, point_rectangle) (recursive call)

---

```
HANDLE_OVERFLOW(node)
```

- Check if node is overflowing
- If node.entries.length <= max_entries:
    - Return (no overflow)
- Split the node
- Call QUADRATIC_SPLIT(node) → returns (node1, node2)
- Handle root split
- If node.parent == null (node is root):
    - Create new root node (internal, not leaf)
  - Add entry (node1.mbr, node1) to new root
  - Add entry (node2.mbr, node2) to new root
  - Set node1.parent = new_root
  - Set node2.parent = new_root
  - Set tree.root = new_root
  - Increment tree.height
  - Return
- Handle non-root split
- Set parent = node.parent
- Add entry (node2.mbr, node2) to parent
- Set node2.parent = parent
- Call HANDLE_OVERFLOW(parent) (propagate upward)

---

```
QUADRATIC_SPLIT(node)
```

- Create new node
- Create new_node with same type (leaf/internal) and max_entries
- Pick seeds - find worst pair
- Initialize max_waste = -1, seed1_idx = 0, seed2_idx = 1
- For each pair (i, j) where i < j in node.entries:
    - Get mbr1 = node.entries[i].rectangle
  - Get mbr2 = node.entries[j].rectangle
  - Calculate union_area = union(mbr1, mbr2).area()
  - Calculate waste = union_area - mbr1.area() - mbr2.area()
  - If waste > max_waste:
        - Set max_waste = waste
    - Set seed1_idx = i, seed2_idx = j
- Initialize groups
- Set group1 = [node.entries[seed1_idx]]
- Set group2 = [node.entries[seed2_idx]]
- Set group1_mbr = node.entries[seed1_idx].rectangle
- Set group2_mbr = node.entries[seed2_idx].rectangle
- Set remaining = all entries except seed1_idx and seed2_idx
- Distribute remaining entries
- While remaining is not empty:
    - Check minimum constraints
        - If group1.length >= min_entries AND remaining.length + group2.length == min_entries:
            - Add all remaining to group2
      - Break
    - If group2.length >= min_entries AND remaining.length + group1.length == min_entries:
            - Add all remaining to group1
      - Break
  - Pick next entry to assign
        - Initialize max_difference = -1, best_entry = null, best_group = 0
    - For each entry in remaining:
            - Get entry_mbr = entry.rectangle
      - Calculate enlarge1 = group1_mbr.enlargement_needed(entry_mbr)
      - Calculate enlarge2 = group2_mbr.enlargement_needed(entry_mbr)
      - Calculate difference = |enlarge1 - enlarge2|
      - If difference > max_difference:
                - Set max_difference = difference
        - Set best_entry = entry
        - Set best_group = 1 if enlarge1 < enlarge2 else 2
  - Assign entry to chosen group
        - If best_group == 1:
            - Add best_entry to group1
      - Update group1_mbr = union(group1_mbr, best_entry.rectangle)
    - Else:
            - Add best_entry to group2
      - Update group2_mbr = union(group2_mbr, best_entry.rectangle)
    - Remove best_entry from remaining
- Update nodes
- Set node.entries = group1
- Set new_node.entries = group2
- If nodes are internal (not leaves):
    - Update parent pointers for all children in both nodes
- Return (node, new_node)

---

```
UPDATE_MBR_UPWARD(node)
```

- If node has no parent
- Return (reached root)
- Update parent's entry
- Calculate new_mbr = node.calculate_mbr()
- For each entry (mbr, child) in parent.entries:
    - If child == node:
        - Update entry to (new_mbr, node)
    - Break
- Continue upward
- Call UPDATE_MBR_UPWARD(node.parent) (recursive)

---

```
CALCULATE_MBR(node)
```

- If node.entries is empty: return null
- Set result = node.entries[0].rectangle
- For each remaining entry in node.entries:
- Set result = union(result, entry.rectangle)
- Return result

```
UNION(rect1, rect2)
```

- Return new Rectangle with:
- x_min = min(rect1.x_min, rect2.x_min)
- y_min = min(rect1.y_min, rect2.y_min)
- x_max = max(rect1.x_max, rect2.x_max)
- y_max = max(rect1.y_max, rect2.y_max)

```
ENLARGEMENT_NEEDED(rect1, rect2)
```
- Calculate union_rect = union(rect1, rect2)
- Return union_rect.area() - rect1.area()

### Example usage

In [168]:
"""RTree usage example."""

from rtree.rtree import RTree

# Create RTree with verbose=True to see tree after each insertion
rtree = RTree(max_entries=3, verbose=True)

"""
  Y
 10 |                                 
  9 |                J                
  8 |                      F          
  7 |             D                   
  6 |                            C    
  5 |          H                      
  4 |                B                
  3 |       A                         
  2 |    G              I             
  1 |                         E       
  0 |                                 
    +---------------------------------
     0  1  2  3  4  5  6  7  8  9 10  X
"""

# Insert some points
points = [
    (2, 3, "Restaurant A"),
    (5, 4, "Restaurant B"),
    (9, 6, "Store C"),
    (4, 7, "Store D"),
    (8, 1, "Park E"),
    (7, 8, "Museum F"),
    (1, 2, "Cafe G"),
    (3, 5, "Library H"),
    (6, 2, "Bank I"),
    (5, 9, "Hospital J"),
]

print("\n" + "=" * 50)
print("INSERTING POINTS INTO R-TREE (with tree visualization after each insertion)")
print("=" * 50)

for x, y, name in points:
    rtree.insert(x, y, name)

print("\n" + "=" * 50)
print("FINAL TREE STRUCTURE:")
print("=" * 50)
print(rtree)

print("\n" + "=" * 50)
print("Searching for points in rectangle (3, 3) to (7, 7):")
results = rtree.search_rectangle(3, 3, 7, 7)
for result in results:
    print(f"  Found: {result}")

print("\n" + "=" * 50)
print("Searching for exact point (5, 4):")
results = rtree.search_point(5, 4)
for result in results:
    print(f"  Found: {result}")

print("\n" + "=" * 50)
print("Searching for KNN for (10, 1) with k = 4:")
results = rtree.knn(10, 0, 4)
for result in results:
    print(f"  Found: {result}")


INSERTING POINTS INTO R-TREE (with tree visualization after each insertion)

📍 Inserted: Restaurant A at (2, 3)
Tree after insertion:
RTree (height=1, max_entries=3)
└── Root Leaf (2,3)
    └── • (2,3): Restaurant A

📍 Inserted: Restaurant B at (5, 4)
Tree after insertion:
RTree (height=1, max_entries=3)
└── Root Leaf [(2,3)-(5,4)]
    ├── • (2,3): Restaurant A
    └── • (5,4): Restaurant B

📍 Inserted: Store C at (9, 6)
Tree after insertion:
RTree (height=1, max_entries=3)
└── Root Leaf [(2,3)-(9,6)]
    ├── • (2,3): Restaurant A
    ├── • (5,4): Restaurant B
    └── • (9,6): Store C

📍 Inserted: Store D at (4, 7)
⚠️  Tree height increased! Rebalancing occurred.
Tree after insertion:
RTree (height=2, max_entries=3)
└── Root Node [(2,3)-(9,7)]
    ├── Leaf [(2,3)-(5,4)]
    │   ├── • (2,3): Restaurant A
    │   └── • (5,4): Restaurant B
    └── Leaf [(4,6)-(9,7)]
        ├── • (9,6): Store C
        └── • (4,7): Store D

📍 Inserted: Park E at (8, 1)
Tree after insertion:
RTree (height

## Array double pointer approach

When an array is sorted, and you need to find 2 numbers that meet a condition.

In [169]:
def sorted_pair_sum(numbers: list[int], target: int) -> list[int]:
    left = 0
    right = len(numbers) - 1

    while left < right:
        # This is the condition to meet, a pair of numbers that is
        # equal to target
        total = numbers[left] + numbers[right]

        if total == target:
            return [left, right]
        elif total > target:
            right -= 1
        else:
            left += 1

    return []


sorted_pair_sum([0, 1, 1, 2, 3, 5, 7, 9], 12)

[4, 7]

But also:

In [170]:
sorted_pair_sum([0, 1, 1, 2, 3, 5, 7, 9], 120)

[]

## Array sliding window to find biggest difference

In [171]:
def biggest_difference(nums: list[int]) -> int:
    left = 0
    right = 1
    biggest_diff = 0

    while right < len(nums):
        # Calculate current difference
        diff = nums[right] - nums[left]

        if diff > 0:
            biggest_diff = max(biggest_diff, diff)
        else:
            # If prices[right] < prices[left],
            # move the left pointer to right
            left = right

        right += 1

    return biggest_diff

print(biggest_difference([7,1,5,3,6,4]))
print(biggest_difference([7,6,4,3,1]))

5
0


## Trees

Given a tree node object:






In [172]:
class TreeNode:
    def __init__(self: Self, val: int, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def __str__(self: Self) -> str:
        return str(self.val)

### Common methods

In [173]:
class TreeUtils:

    def __init__(self: Self):
        pass

    def height(self: Self, node: TreeNode) -> int:
        self.diameter = 0

        # Base case: empty node has height 0
        if not node:
            return 0

        # Recursively get heights of subtrees
        left_height = self.height(node.left)
        right_height = self.height(node.right)

        # Update diameter: path through this node
        # is left_height + right_height
        self.diameter = max(self.diameter, left_height + right_height)

        # Return height of this subtree
        # Height = 1 + max(left_height, right_height)
        return 1 + max(left_height, right_height)

    def lowest_common_ancestor(self: Self, root: 'TreeNode', p: TreeNode, q: TreeNode) -> TreeNode:
        # Break case:
        if not root or root.val == p.val or root.val == q.val:
            return root

        # We can treat left and right as "roots" and call
        # recursively because it will reach a point where
        # it will reach the bottom.
        left = self.lowest_common_ancestor(root.left, p, q)
        right = self.lowest_common_ancestor(root.right, p, q)

        # If both sides return non-None, the current node is LCA
        if left and right:
            return root

        # Otherwise, return whichever side is non-null
        return left if left else right


tree_utils = TreeUtils()


### Height

In [174]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

tree_utils.height(root)

3

### Lowest common ancestor

In [175]:
root = TreeNode(3)

root.left = TreeNode(5)
root.left.left = TreeNode(6)
root.left.right = TreeNode(2)
root.left.right.left = TreeNode(7)
root.left.right.right = TreeNode(4)

root.right = TreeNode(1)
root.right.left = TreeNode(0)
root.right.right = TreeNode(8)

print(tree_utils.lowest_common_ancestor(root, TreeNode(5), TreeNode(1)))

3


## Min Heap

A min heap is a complete binary tree where every parent node is smaller than or equal to its children. The smallest element is always at the root.

### Visual Representation

```
        1           (root - smallest)
       / \
      3   2
     / \ / \
    7  4 5  6
```
Array representation: [1, 3, 2, 7, 4, 5, 6]

### Key Properties

* Complete Binary Tree: All levels filled except possibly the last, which fills left-to-right
* Min Heap Property: Parent ≤ Children
* Root is Minimum: The smallest element is always at index 0
* No Ordering Between Siblings: Only parent-child relationship matters

### Array Representation
Heaps are typically stored in arrays where for any element at index i:

* Parent: `(i-1) // 2`
* Left child: `2*i + 1`
* Right child: `2*i + 2`

### Using it to find Kth largest element

In [176]:
def find_kth_largest(nums: list[int], k: int) -> int:
    # Create a min-heap with first k elements
    heap = nums[:k]
    heapq.heapify(heap)

    # Process remaining elements
    for num in nums[k:]:
        if num > heap[0]:  # If the current element is larger than smallest in heap
            heapq.heapreplace(heap, num)  # Remove smallest, add current

    # The root of min-heap is the kth largest
    return heap[0]


find_kth_largest([3, 2, 3, 1, 2, 4, 5, 5, 6], 4)

4

## Binary search to find a missing number in an array

In [177]:
def find_kth_positive(arr: list[int], k: int) -> int:
    # Binary search to find position
    left, right = 0, len(arr)

    while left < right:
        mid = left + (right - left) // 2
        if arr[mid] - mid - 1 < k:
            left = mid + 1
        else:
            right = mid

    # Direct formula: the answer is always k + left
    return k + left


print(find_kth_positive([2,3,4,7,11], 5))
print(find_kth_positive([1,2,3,4], 2))

9
6
